In [1]:
import mwu_measures.processing_corpus

mwu_measures.processing_corpus.process_corpus(corpus='bnc', corpus_dir='small_corpus.txt', verbose=False, chunk_size=100000)

Merging....


In [2]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from multiprocessing import cpu_count
# from multiprocessing import Pool, cpu_count
# from itertools import repeat
from joblib import Parallel, delayed, parallel_config
from functools import partial
from mwu_measures.compute_functions import min_max_norm
from mwu_measures import compute_functions
from mwu_measures import processing_corpus
CONSOLIDATED_FW = None
CONSOLIDATED_BW = None

In [11]:
ngram = ('in the world')

In [12]:
"""
Function for computing the MWU measures for a target ngram. 
:param ngram: A string with the ngram to be analyzed.
:returns: A dictionary with all MWU measures obtained:
    Token frequency, dispersion, type frequency for each slot,
    entropy difference for each slot, both directions of 
    association.
"""
comps = ngram.split(' ')
comp_1 = comps[0]
comp_2 = comps[1]
comp_3 = comps[2]

In [13]:
forward_dict = processing_corpus.TRIGRAM_FW
backward_dict = processing_corpus.TRIGRAM_BW
backward_merged = processing_corpus.TRIGRAM_MERGED_BW
corpus_proportions = processing_corpus.CORPUS_PROPORTIONS

In [14]:
trigram_freq = [(corpus, corpus_dict.get(comp_1, pd.Series(0)).get(comp_2, pd.Series(0)).get(comp_3, 0)) for
    corpus, corpus_dict in forward_dict.items()]
token_freq = sum(freqs[1] for freqs in trigram_freq)

In [15]:
def get_dispersion(ngram, token_freq, corpus_proportions):
    """
    Computes the "Dispersion" variable for an ngram as the
    KLD divergence between its occurrences in each corpus and
    the overall corpus proportions.
    :param ngram: The frequency of the ngram in each
        corpus as a (corpus, frequency) tuple.
    :param token_freq: The token frequency of the ngram,
        used to transform frequencies into proportions.
    :returns: Dispersion measure as a scalar.
    """
    if not isinstance(corpus_proportions, pd.DataFrame):
        corpus_proportions = pd.DataFrame.from_dict(
            corpus_proportions, orient='index'
        ).reset_index().rename(
            columns={'index': 'corpus', 0: 'corpus_prop'}
            )
    ngram_props = [(corpus, freq / token_freq) for corpus, freq in ngram]
    ngram_props = pd.DataFrame(ngram_props, columns=['corpus', 'ngram_prop'])
    ngram_props = pd.merge(
        corpus_proportions,
        ngram_props,
        on='corpus',
        how='left'
        ).fillna(0)
    kld_props = compute_functions.get_kld(ngram_props['ngram_prop'].values,
        ngram_props['corpus_prop'].values)
    return kld_props
# works
dispersion = get_dispersion(trigram_freq, token_freq, corpus_proportions)

In [ ]:
freqs_ngram_1 = [corpus_dict.get(comp_1, pd.Series(0)).get(comp_2, pd.Series(0)) for corpus_dict in forward_dict.values()]
freqs_ngram_1 = sum(freqs_ngram_1, Counter())
freqs_ngram_2 = [corpus_dict.get(comp_3, pd.Series(0)) for corpus_dict in backward_merged.values()]
freqs_ngram_2 = sum(freqs_ngram_2, Counter())

In [ ]:
# Ideally, same function for bigram and trigram with an if somewhere for this part
freqs_ngram_1 = [Counter(dict(corpus_dict.get(comp_1, pd.Series(0)))) for corpus_dict in forward_dict.values()]
freqs_ngram_2 = [FreqDist(dict(corpus_dict.get(comp_2, pd.Series(0)))) for corpus_dict in backward_dict.values()]
# freqs_ngram_1 = sum(freqs_ngram_1, FreqDist())
# freqs_ngram_2 = sum(freqs_ngram_2, FreqDist())

In [43]:
trigram_freq = [(corpus, corpus_dict.get(comp_1, pd.Series(0)).get(comp_2, pd.Series(0))).get(comp_3) for
    corpus, corpus_dict in forward_dict.items()]
print(f'<<{" ".join([comp_1, comp_2, comp_3])}>> is not in the corpus')

AttributeError: 'tuple' object has no attribute 'get'

In [ ]:
    """
    Function for computing the MWU measures for a target ngram. 
    :param ngram: A string with the ngram to be analyzed.
    :returns: A dictionary with all MWU measures obtained:
        Token frequency, dispersion, type frequency for each slot,
        entropy difference for each slot, both directions of 
        association.
    """
    comps = ngram.split(' ')
    comp_1 = comps[0]
    comp_2 = comps[1]
    # forward_dict = processing_corpus.BIGRAM_FW
    # backward_dict = processing_corpus.BIGRAM_BW
    bigram_freq = [(corpus, corpus_dict.get(comp_1, pd.Series(0)).get(comp_2, 0)) for
        corpus, corpus_dict in forward_dict.items()]
    # Token frequency
    token_freq = sum(freqs[1] for freqs in bigram_freq)
    if token_freq == 0:
        if verbose:
            print(f'<<{" ".join([comp_1, comp_2, comp_3])}>> is not in the corpus')
        return None
    # Dispersion
    dispersion = get_dispersion(bigram_freq, token_freq, corpus_proportions)

    ## Need aggregate frequencies for the rest
    freqs_ngram_1 = [FreqDist(dict(corpus_dict.get(comp_1, pd.Series(0)))) for corpus_dict in forward_dict.values()]
    freqs_ngram_2 = [FreqDist(dict(corpus_dict.get(comp_2, pd.Series(0)))) for corpus_dict in backward_dict.values()]
    freqs_ngram_1 = sum(freqs_ngram_1, FreqDist())
    freqs_ngram_2 = sum(freqs_ngram_2, FreqDist())